# Welcome back Commander!

This is part 2, if you haven't covered [part 1](https://www.kaggle.com/bovard/kore-intro-i-the-basics/) yet, give it a look!

In part 2, we'll look at how to mine Kore, and learn more about flight paths.

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

## Mining

In Kore Fleets, a fleet picks up a certain % of the kore on the square it occupies at the end of its turn. This relationship is logrithmic, so many smaller fleets will pick up more kore than one large one. See the table below:

| Number Ships | % mined  |
| --- | --- | 
| 1 | 0% |
| 2 | 3% |
| 3 | 5% |
| 5 | 8% |
| 8 | 10% |
| 13 | 13% |
| 21 | 15% |
| 34 | 18% |
| 55 | 20% |
| 91 | 23% |
| 149 | 25% |
| 245 | 28% |
| 404 | 30% |

The exact formula to get the % mined is `ln(num_ships_in_fleet) / 20`.

For example, `ln(55) / 20 = .20037`

Let's look at an example, 4 smaller fleets with 8 ships each vs one flet with 32 ships. For simplicites sake, let's ignore the 2% regrowth a turn.

```
Kore Mined, 4 fleets of 8 (10% mining rate)
initial_kore = 100
after first 8-ship fleet => 90
after second 8-ship fleet => 81
after third 8-ship fleet => 72.9
after fourth 8-ship fleet => 65.6

Total Kore mined 100 - 65.6 = 34.4

Kore Mined, 1 fleet of 32 (17% mining rate)
initial_kore = 100
after first 32-ship fleet => 83

Total Kore mined 100 - 83 = 17
```

So the smaller fleets mined 2x more!

## The Downside of smaller fleets

The downside of smaller fleets is that they can have shorter flight plan instructions! The `length` of a flight plan is equal to the length of the string representing it.

For example, the flight plan "go north, continue 8 spaces, then to south", represented as `"N8S"` has a length of 3. A more complicated flight plan that goes in a loop, "N8E8S8W" (note you don't need a trailing 8), has length 7.

The below table shows the largest flight instructions that can be given to a fleet.

| Number Ships | Max Flight Plan Length  |
| --- | --- | 
| 1 | 1 |
| 2 | 2 |
| 3 | 3 |
| 5 | 4 |
| 8 | 5 |
| 13 | 6 |
| 21 | 7 |
| 34 | 8 |
| 55 | 9 |
| 91 | 10 |
| 149 | 11 |
| 245 | 12 |
| 404 | 13 |

Observant readers will notice these are an approximation of the [Fibbonaci numbers](https://en.wikipedia.org/wiki/Fibonacci_number), and and this is given by the formula `floor(2 * ln(num_ships)) + 1`[](http://)

## Putting them together

Large fleets are more manuverable, but don't mine quickly, while smaller fleets mine quickly, but are more limited. To take advantage of both, use the *fleet coalescence* mechanic!

> Any allied fleets that currently occupy the same space are added to the largest allied fleet in that location. Ship size, current kore, and finally direction (NESW) are used to determine the largest fleet.

This means when two or more allied fleets end up occupying the same square, they will join the larger one (in terms of ships)! So your bigger fleets can "pick up" your smaller fleets after they are done mining.

Let's see if we can code a simple example of this!

In [ ]:
%%writefile miner.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    period = 4 + config.size + 1
    
    for shipyard in me.shipyards:
        action = None
        if turn % period == 4:
            action = ShipyardAction.launch_fleet_with_flight_plan(2, "ES")
        elif turn % period == 6: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2S")
        elif turn % period == 4 + config.size:
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3W")
            shipyard.next_action = action
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/miner.py"])
env.render(mode="ipython", width=1000, height=800)

That works! But it's very slow. A better version might be a box miner, let's give that a shot!

In [ ]:
%%writefile box_miner.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    period = 40
    
    for shipyard in me.shipyards:
        action = None
        if turn < 40:
            action = ShipyardAction.spawn_ships(1)
        elif turn % period == 1:
            action = ShipyardAction.launch_fleet_with_flight_plan(21, "E9N9W9S")
        elif turn % period == 3: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E8N")
        elif turn % period == 5: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E7N")
        elif turn % period == 7: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E6N")
        elif turn % period == 9: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E5N")
        elif turn % period == 11: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E4N")
        elif turn % period == 13: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3N")
        elif turn % period == 15: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2N")
        elif turn % period == 17: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E1N")
        elif turn % period == 19: 
            action = ShipyardAction.launch_fleet_with_flight_plan(2, "EN")
        elif turn % period == 21: 
            action = ShipyardAction.launch_fleet_with_flight_plan(2, "N")
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/box_miner.py"])
env.render(mode="ipython", width=1000, height=800)

### Done!

And there we have it! Enjoy the show and happy coding!

Next we'll learn how to [expand your empire](https://www.kaggle.com/bovard/kore-intro-iii-expanding-the-empire) by building more shipyards.